This notebook is a test of the aiida-aimall workchains using the aiida-submission-controller. We will use 100 cml files as the test case and configure the controllers as we please. This assumes that codes and computers have been setup already. It starts by loading aiida using the default profile, and creating all the needed groups (if they do not already exist), along with the usual import statements.

It then creates the cml list as a subset of the cml files in the Data subdirectory in this folder, and defines all inputs needed for the workchains. It creates the workchains and the four necessary controllers, then runs a loop to continually check the processes for new submissions.

In [1]:
#can run this as notebook
#load AiiDA, using the default profile, which you should have set to be the correct one
%load_ext aiida
%aiida
#Create groups needed - the folders for our nodes
# The controllers defined below work like this: They look for the entries in a given folder.
# The given folder is the one containing the inputs to the next step of the chain:
#   inp_frag for G16FragController - contains structures of fragments
#   opt_wfx for AIMReorController - contains wfx files from optimization calculations
#   reor_structs for GaussianSubmissionController - contains reoriented structures
#   reor_wfx for AimAllSubmissionController - contains wfx from the single points on reoriented structures
# The controllers also store their output in folders, will be useful for querying later
%verdi group create aim_reor
%verdi group create g16_opt
%verdi group create gaussian_sp
%verdi group create inp_frag
%verdi group create opt_workchain
%verdi group create opt_wfx
%verdi group create prereor_aim
%verdi group create reor_wfx
%verdi group create reor_structs

Report: Group with label 'aim_reor' already exists: Group<aim_reor>.
Report: Group with label 'g16_opt' already exists: Group<g16_opt>.
Report: Group with label 'gaussian_sp' already exists: Group<gaussian_sp>.
Report: Group with label 'inp_frag' already exists: Group<inp_frag>.
Report: Group with label 'opt_workchain' already exists: Group<opt_workchain>.
Report: Group with label 'opt_wfx' already exists: Group<opt_wfx>.
Report: Group with label 'prereor_aim' already exists: Group<prereor_aim>.
Report: Group with label 'reor_wfx' already exists: Group<reor_wfx>.
Report: Group with label 'reor_structs' already exists: Group<reor_structs>.


In [1]:
#imports
from aiida.engine import submit # submit the workchain
import time # delays the while loop
import os
from aiida.orm import Dict # Data type
from aiida.plugins.factories import  WorkflowFactory # load workflows
# load controllers
from aiida_aimall.controllers import AimAllSubmissionController, AimReorSubmissionController, GaussianSubmissionController, G16FragController
# load the first workchain
MultiFragmentWorkChain = WorkflowFactory('aimall.multifrag')

#Restart the daemons just to make sure they are on
%verdi daemon stop
%verdi daemon start 5
# while running in a terminal, us verdi process list to see running workflows.
# At the bottom of that report, see the usage of daemons. You may need to start some more daemons depending
# on what you set as the number of chains to run
%verdi status

builder = MultiFragmentWorkChain.get_builder()
cfd = {}
#get 100 cml files in the cfd dictionary
with open(os.getcwd() + 'Data/cml_files.txt','r') as file:
    file_string = file.readlines()
    for i,line in enumerate(file_string):
        if i < 100:
            line = line.replace('\n','')
            line = line.replace('./',os.getcwd() + 'Data/')
            cfd[str(i)] = line
cml_dict = Dict(dict = cfd)
# create fragmenting parameters
frag_params = Dict({'input':'deprecated, here just to not rock boat currently',
                    'bb_patt':'[$([C;X4;!R]):1]-[$([R,!$([C;X4]);!#0;!#9;!#17;!#35;!#1]):2]',
                    'keep_only_children':True,
                    'cml_file':'',
                    'include_parent':True,
                    'input_type':'cmlfile'})

# pass the inputs to the fragmenting workchain
builder.frag_params = frag_params
builder.cml_file_dict = cml_dict

#Gaussian optimization parameters
parameters = Dict(dict={
        'link0_parameters': {
            '%chk':'aiida.chk',
            "%mem": "3200MB", # Currently set to use 8000 MB in .sh files
            "%nprocshared": 4,
        },
        'functional':'wb97xd',
        'basis_set':'aug-cc-pvtz',
        'charge': 0,
        'multiplicity': 1,
        'route_parameters': {'opt': None, 'Output':'WFX'},
        "input_parameters": {"output.wfx": None},
    })


# Gaussian optimization controller
g16opt_controller = G16FragController(
    parent_group_label = "inp_frag", # group to look for fragment structures
    group_label = "opt_workchain", # group to store opt workchains
    max_concurrent = 10, # number of concurrent calculations, don't set too high, don't want to overload cluster
    code_label = "gaussian@cedar",
    g16_opt_params = parameters.get_dict() # for creating the Gaussian input file
)

# AIM Reor Controller
controller = AimReorSubmissionController(
    parent_group_label = 'opt_wfx',
    group_label = 'prereor_aim',
    max_concurrent = 1, # set to 1 since we will use 2 processors, and the second AIM controller will also use 2
    #So max 1 of each AIM  controller at a time=2 concurrent AIM
    code_label='aimall@localhost'
)

# Gaussian Single Point parameters 
sp_parameters = Dict(dict={
        'link0_parameters': {
            '%chk':'aiida.chk',
            "%mem": "4000MB",
            "%nprocshared": 4,
        },
        'functional':'wb97xd',
        'basis_set':'aug-cc-pvtz',
        'charge': 0,
        'multiplicity': 1,
        'route_parameters': {'nosymmetry':None, 'Output':'WFX'},
        "input_parameters": {"output.wfx": None},
        
    })
#Gaussian single point controller
controller2 = GaussianSubmissionController(
    parent_group_label = 'reor_structs',
    group_label = 'gaussian_sp',
    max_concurrent = 10,
    code_label='gaussian@cedar',
    g16_sp_params=sp_parameters
)

# Final AIM Controller
controller3 = AimAllSubmissionController(
    code_label='aimall@localhost',
    parent_group_label = 'reor_wfx',
    group_label = 'aim_reor',
    max_concurrent = 1,
    aim_parser = 'aimqb.group'
)

UsageError: Line magic function `%verdi` not found.


In [4]:
submit(builder) # launch the fragmenting
#IMPORTANT!!!
# Wait until this workchain is DONE before continuing

<WorkChainNode: uuid: c811e723-b70e-4b9e-a359-a9b204df825b (pk: 6) (aiida.workflows:multifrag)>

In [4]:
# loop over submitting in batches every interval until all are run
from aiida.engine.processes.control import play_processes
while controller3.num_already_run < g16opt_controller.num_to_run + g16opt_controller.num_already_run:
    # Submit Gaussian batches every hour. 12 AIM loops * 5 min
    g16opt_controller.submit_new_batch()
    print(f'Opt Freq Controller {g16opt_controller.num_already_run}')
    i=0
    play_processes(all_entries=True)
    while i < 12:
        #submit AIM batches every 5 minutes
        i = i+1
        controller.submit_new_batch()
        print(f'AimReor Controller {controller.num_already_run}')
        controller2.submit_new_batch()
        print(f'Gaussian SP Controller {controller2.num_already_run}')
        print(f'Final AIM Controller {controller3.num_already_run}')
        controller3.submit_new_batch()
        time.sleep(300)

Opt Freq Controller 29
Report: no active processes selected.


REPORT:aiida.process_control:no active processes selected.


AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 16
Gaussian SP Controller 5
Final AIM Controller 1
Opt Freq Controller 29
Report: no active processes selected.


REPORT:aiida.process_control:no active processes selected.


AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
AimReor Controller 17
Gaussian SP Controller 5
Final AIM Controller 1
Opt Freq Controller 29
Report: no active processes selected.


...

AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
Opt Freq Controller 48
Report: no active processes selected.


REPORT:aiida.process_control:no active processes selected.


AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
Opt Freq Controller 48
Report: no active processes selected.


REPORT:aiida.process_control:no active processes selected.


AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
AimReor Controller 37
Gaussian SP Controller 16
Final AIM Controller 14
